In [1]:
import chromadb

from langchain_community.embeddings import OllamaEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Chroma
from langchain_community.llms import Ollama    

In [2]:
db = Chroma(
        persist_directory="./chroma",
        embedding_function=HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-small",
            model_kwargs={'device': 'cpu'}
        )
    )

c:\Users\50272\anaconda3\envs\law_chatbot\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\50272\anaconda3\envs\law_chatbot\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\50272\anaconda3\envs\law_chatbot\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated vers

In [3]:
def create_agent_chain():
    model_name = "aya"
    llm = Ollama(model=model_name)
    chain = load_qa_chain(llm, chain_type="stuff")
    return chain

In [10]:
PROMPT_TEMPLATE = """
Anda adalah seorang paralegal hukum AI. Anda akan diberikan konteks
untuk menjawab pertanyaan yang diberikan. Jika jawaban dari pertanyaan yang diberikan
tidak ada pada konteks, cukup jawab dengan "Maaf, saya tidak tahu tentang hal itu" dan jangan membuat
jawaban sendiri. Jawaban yang anda berikan harus jelas dan minimal terdiri dari 2 paragraf.
Berikut adalah konteks yang harus anda jadikan acuan untuk menjawab pertanyaan:

{context}

---

Jawab pertanyaan berikut berdasarkan konteks: {question}
"""

In [13]:
from langchain.prompts import ChatPromptTemplate

def get_llm_response(query):
    chain = create_agent_chain()
    result = db.similarity_search(query, k=5)

    context = "\n\n---\n\n".join([doc.page_content for doc in result])

    template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

    prompt = template.format(context=context, question=query)

    print(prompt)

    answer = chain.invoke({
        "input_documents": result,
        "question": prompt
    })
    
    response = {
        "response": answer['output_text'],
    }

    return response

In [11]:
retriever = db.as_retriever()

def get_llm_response(query):
    # Gunakan retriever untuk mencari dokumen relevan
    result = retriever.get_relevant_documents(query, k=5)

    # Ambil konten dokumen dari hasil retriever
    context = "\n\n---\n\n".join([doc.page_content for doc in result])
    
    # Siapkan template prompt
    template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = template.format(context=context, question=query)
    
    # Debugging: tampilkan prompt untuk memastikan format yang benar
    print(prompt)

    # Jalankan model untuk mendapatkan jawaban
    chain = create_agent_chain()
    answer = chain.invoke({
        "input_documents": result,
        "question": prompt
    })
    
    # response = {
    #     "response": answer['output_text'],
    # }

    return answer

In [14]:
query = "Apa inti dari pembukaan UUD 1945 Bait ke dua?"

get_llm_response(query)

Human: 
Anda adalah seorang paralegal hukum AI. Anda akan diberikan konteks
untuk menjawab pertanyaan yang diberikan. Jika jawaban dari pertanyaan yang diberikan
tidak ada pada konteks, cukup jawab dengan "Maaf, saya tidak tahu tentang hal itu" dan jangan membuat
jawaban sendiri. Jawaban yang anda berikan harus jelas dan minimal terdiri dari 2 paragraf.
Berikut adalah konteks yang harus anda jadikan acuan untuk menjawab pertanyaan:

UNDANG -UNDANG DASAR NEGARA REPUBLIK INDONESIA
TAHUN 1945
PEMBUKAAN
( P r e a m b u l e )
Bahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,
maka penjajahan di atas dunia harus dihapuskan, karena tida k sesuai dengan peri -
kemanusiaan dan peri -keadilan.
Dan perjuangan pergerakan kemerdekaan Indonesia telah sam pailah kepada saat
yang berbahagia dengan selamat sentausa mengantarkan raky at Indonesia ke depan
pintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil
dan makmur.
Atas berkat rakhmat Allah Y

{'response': 'Maaf, saya tidak tahu tentang hal itu.'}

In [8]:
query2 = "Apa saja kewenangan dan tugas Komisi Yudisial?"

get_llm_response(query2)

Human: 
Anda adalah seorang paralegal hukum AI. Anda akan diberikan konteks
untuk menjawab pertanyaan yang diberikan. Jika jawaban dari pertanyaan yang diberikan
tidak ada pada konteks, cukup jawab dengan "Maaf, saya tidak tahu tentang hal itu" dan jangan membuat
jawaban sendiri. Jawaban yang anda berikan harus jelas dan minimal terdiri dari 2 paragraf.
Jawab pertanyaan yang diberikan hanya berdasarkan konteks berikut:

TAMBAHAN 
LEMBARAN NEGARA R.I 
No.6124   KEUANGAN Negara. Hak Keuangan. Fasilitas. Komisi 
Yudisial. (Penjelasan atas Lembaran Negara Republik 
Indonesia Tahun 2017 Nomor 207) 
 
PENJELASAN 
ATAS 
PERATURAN PEMERINTAH REPUBLIK INDONESIA 
NOMOR 39 TAHUN 2017 
TENTANG 
HAK KEUANGAN DAN FASILITAS ANGGOTA KOMISI YUDISIAL 
 
 
I. UMUM 
Pasal 24B Undang-Undang Dasar Negara Republik Indonesia  Tahun 
1945 juncto  Pasal 20 Undang-Undang Nomor 22 Tahun 2004 sebagaim ana 
telah diubah dengan Undang-Undang Nomor 18 Tahun 2011 tentang 
Komisi Yudisial ditegaskan peranan penting Kom

{'input_documents': [Document(metadata={'page': 0, 'source': '/content/PP/2017/Peraturan Pemerintah Nomor 39 Tahun 20170_fix.pdf'}, page_content='TAMBAHAN \nLEMBARAN NEGARA R.I \nNo.6124   KEUANGAN Negara. Hak Keuangan. Fasilitas. Komisi \nYudisial. (Penjelasan atas Lembaran Negara Republik \nIndonesia Tahun 2017 Nomor 207) \n \nPENJELASAN \nATAS \nPERATURAN PEMERINTAH REPUBLIK INDONESIA \nNOMOR 39 TAHUN 2017 \nTENTANG \nHAK KEUANGAN DAN FASILITAS ANGGOTA KOMISI YUDISIAL \n \n \nI. UMUM \nPasal 24B Undang-Undang Dasar Negara Republik Indonesia  Tahun \n1945 juncto  Pasal 20 Undang-Undang Nomor 22 Tahun 2004 sebagaim ana \ntelah diubah dengan Undang-Undang Nomor 18 Tahun 2011 tentang \nKomisi Yudisial ditegaskan peranan penting Komisi Yudisial dalam  usaha \nuntuk menjaga dan menegakkan kehormatan, keluhuran ma rtabat, serta \nperilaku Hakim, demi tegaknya hukum dan keadilan sesuai de ngan \nUndang-Undang Dasar Negara Republik Indonesia Tahun 1945 , serta \ndemi mewujudkan kekuasaan keh

In [9]:
query3 = "Siapa kiper terbaik timnas Indonesia sepanjang masa?"

get_llm_response(query3)

Human: 
Anda adalah seorang paralegal hukum AI. Anda akan diberikan konteks
untuk menjawab pertanyaan yang diberikan. Jika jawaban dari pertanyaan yang diberikan
tidak ada pada konteks, cukup jawab dengan "Maaf, saya tidak tahu tentang hal itu" dan jangan membuat
jawaban sendiri. Jawaban yang anda berikan harus jelas dan minimal terdiri dari 2 paragraf.
Jawab pertanyaan yang diberikan hanya berdasarkan konteks berikut:

No.6065 -3- 
Pasal 10  
Cukup jelas. 
 
 
 
www.peraturan.go.id

---

2021 , No.26 -2010- 
www.peraturan.go.id

---

TAMBAHAN 
LEMBARAN NEGARA R.I 
No.6061 HANKAM. Pencarian dan Pertolongan. Operasi. 
(Penjelasan atas Lembaran Negara Republik Indonesia 
Tahun 2017 Nomor 113) 
 
PENJELASAN 
ATAS 
PERATURAN PEMERINTAH REPUBLIK INDONESIA 
NOMOR 22 TAHUN 2017 
TENTANG 
OPERASI PENCARIAN DAN PERTOLONGAN 
 
 
I. UMUM 
Indonesia merupakan negara kepulauan terbesar di dunia yang 
memiliki sekitar 17.508 (tujuh belas ribu lima ratus delapan) pulau 
dengan total luas wilayah kura

{'input_documents': [Document(metadata={'page': 2, 'source': '/content/PP/2017/Peraturan Pemerintah Nomor 26 Tahun 20170_fix.pdf'}, page_content='No.6065 -3- \nPasal 10  \nCukup jelas. \n \n \n \nwww.peraturan.go.id'),
  Document(metadata={'page': 2009, 'source': '/content/PP/2021/Peraturan Pemerintah Nomor 16 Tahun 20210_fix.pdf'}, page_content='2021 , No.26 -2010- \nwww.peraturan.go.id'),
  Document(metadata={'page': 0, 'source': '/content/PP/2017/Peraturan Pemerintah Nomor 22 Tahun 20170_fix.pdf'}, page_content='TAMBAHAN \nLEMBARAN NEGARA R.I \nNo.6061 HANKAM. Pencarian dan Pertolongan. Operasi. \n(Penjelasan atas Lembaran Negara Republik Indonesia \nTahun 2017 Nomor 113) \n \nPENJELASAN \nATAS \nPERATURAN PEMERINTAH REPUBLIK INDONESIA \nNOMOR 22 TAHUN 2017 \nTENTANG \nOPERASI PENCARIAN DAN PERTOLONGAN \n \n \nI. UMUM \nIndonesia merupakan negara kepulauan terbesar di dunia yang \nmemiliki sekitar 17.508 (tujuh belas ribu lima ratus delapan) pulau \ndengan total luas wilayah kurang 